<a href="https://colab.research.google.com/github/spaceo521/Data-Science-Projects/blob/main/Assignment_week3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Problem Statement
###How good is player X against CSK in Chepauk when they’re playing 3 spinners

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',None)
pd.set_option('display.expand_frame_repr',False)
pd.set_option('max_colwidth',None)

In [ ]:
deliveries= pd.read_csv('deliveries_updated_mens_ipl.csv')
matches=pd.read_csv('matches_updated_mens_ipl.csv')

In [ ]:
df1=deliveries.copy()
df2=matches.copy()

In [ ]:
df1.head(1)

,matchId,inning,over_ball,over,ball,batting_team,bowling_team,batsman,non_striker,bowler,batsman_runs,extras,isWide,isNoBall,Byes,LegByes,Penalty,dismissal_kind,player_dismissed
0,1082591,1,0.1,0,1,Sunrisers Hyderabad,Royal Challengers Bangalore,DA Warner,S Dhawan,TS Mills,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def balls_per_dismissal(balls,dismissals):
    if dismissals > 0:
        return balls/dismissals
    else:
        return balls/1

def balls_per_boundary(balls,boundaries):
    if boundaries > 0:
        return balls/boundaries
    else:
        return balls/1

In [ ]:
def phase(over):
    if over<=6:
        return 'Powerplay'
    elif over<=15:
        return 'Middle'
    else:
        return 'Death'

In [ ]:
df1['Batting_Phase'] = df1['over'].apply(lambda x: phase(x))


In [ ]:
df1.head()

,matchId,inning,over_ball,over,ball,batting_team,bowling_team,batsman,non_striker,bowler,batsman_runs,extras,isWide,isNoBall,Byes,LegByes,Penalty,dismissal_kind,player_dismissed,Batting_Phase
0,1082591,1,0.1,0,1,Sunrisers Hyderabad,Royal Challengers Bangalore,DA Warner,S Dhawan,TS Mills,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Powerplay
1,1082591,1,0.2,0,2,Sunrisers Hyderabad,Royal Challengers Bangalore,DA Warner,S Dhawan,TS Mills,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Powerplay
2,1082591,1,0.3,0,3,Sunrisers Hyderabad,Royal Challengers Bangalore,DA Warner,S Dhawan,TS Mills,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Powerplay
3,1082591,1,0.4,0,4,Sunrisers Hyderabad,Royal Challengers Bangalore,DA Warner,S Dhawan,TS Mills,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Powerplay
4,1082591,1,0.5,0,5,Sunrisers Hyderabad,Royal Challengers Bangalore,DA Warner,S Dhawan,TS Mills,0,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,Powerplay


#Performance analysis by Innings

In [ ]:
def ByInnings(df,current_innings):
  df=df[df.inning==current_innings]
  df.reset_index(inplace=True,drop=True)
  df['isDot'] = df['batsman_runs'].apply(lambda x : 1 if x == 0 else 0)
  df['isOne'] = df['batsman_runs'].apply(lambda x : 1 if x == 1 else 0)
  df['isTwo'] = df['batsman_runs'].apply(lambda x : 1 if x == 2 else 0)
  df['isThree'] = df['batsman_runs'].apply(lambda x : 1 if x == 3 else 0)
  df['isFour'] = df['batsman_runs'].apply(lambda x :1 if x == 4 else 0)
  df['isSix'] = df['batsman_runs'].apply(lambda x : 1 if x == 6 else 0)
  runs = pd.DataFrame(df.groupby(df['batsman'])['batsman_runs'].sum()).reset_index().rename(columns ={'batsman_runs':'runs'})
  balls = pd.DataFrame(df.groupby(['batsman'])['batsman_runs'].count()).reset_index().rename(columns = {'batsman_runs':'ballsFaced'})
  dismissals = pd.DataFrame(df.groupby(['batsman'])['player_dismissed'].count()).reset_index().rename(columns ={'player_dismissed':'dismissals'})
  innings = pd.DataFrame(df.groupby(['batsman'])['matchId'].apply(lambda x : len(list(np.unique(x))))).reset_index().rename(columns = {'matchId':'innings'})
  dots = pd.DataFrame(df.groupby(['batsman'])['isDot'].sum()).reset_index().rename(columns ={'isDot':'dots'})
  fours = pd.DataFrame(df.groupby(['batsman'])['isFour'].sum()).reset_index().rename(columns ={'isFour':'fours'})
  sixes = pd.DataFrame(df.groupby(['batsman'])['isSix'].sum()).reset_index().rename(columns ={'isSix':'sixes'})
  df=pd.merge(innings,runs,on ='batsman').merge(balls,on='batsman').merge(dismissals,on='batsman').merge(dots,on='batsman').merge(fours,on='batsman').merge(sixes,on='batsman')

  #StrikeRate
  df['SR'] = df.apply(lambda x: 100*(x['runs']/x['ballsFaced']), axis = 1)
  #runs per innings
  df['RPI'] = df.apply(lambda x: x['runs']/x['innings'], axis = 1)

  #balls per dismissals
  df['BPD'] = df.apply(lambda x: balls_per_dismissal(x['ballsFaced'], x['dismissals']), axis = 1)

  #balls per boundary
  df['BPB'] = df.apply(lambda x: balls_per_boundary(x['ballsFaced'], (x['fours'] + x['sixes'])), axis = 1)

  return df


#Phases Of Play

In [ ]:
def Byphase(df,current_phase):
  df=df[df['Batting_Phase'] == current_phase]
  df.reset_index(inplace=True,drop=True)
  df['isDot'] = df['batsman_runs'].apply(lambda x : 1 if x == 0 else 0)
  df['isOne'] = df['batsman_runs'].apply(lambda x : 1 if x == 1 else 0)
  df['isTwo'] = df['batsman_runs'].apply(lambda x : 1 if x == 2 else 0)
  df['isThree'] = df['batsman_runs'].apply(lambda x : 1 if x == 3 else 0)
  df['isFour'] = df['batsman_runs'].apply(lambda x :1 if x == 4 else 0)
  df['isSix'] = df['batsman_runs'].apply(lambda x : 1 if x == 6 else 0)

  runs = pd.DataFrame(df.groupby(df['batsman'])['batsman_runs'].sum()).reset_index().rename(columns ={'batsman_runs':'runs'})
  balls = pd.DataFrame(df.groupby(['batsman'])['batsman_runs'].count()).reset_index().rename(columns = {'batsman_runs':'ballsFaced'})
  dismissals = pd.DataFrame(df.groupby(['batsman'])['player_dismissed'].count()).reset_index().rename(columns ={'player_dismissed':'dismissals'})
  innings = pd.DataFrame(df.groupby(['batsman'])['matchId'].apply(lambda x : len(list(np.unique(x))))).reset_index().rename(columns = {'matchId':'innings'})
  dots = pd.DataFrame(df.groupby(['batsman'])['isDot'].sum()).reset_index().rename(columns ={'isDot':'dots'})
  fours = pd.DataFrame(df.groupby(['batsman'])['isFour'].sum()).reset_index().rename(columns ={'isFour':'fours'})
  sixes = pd.DataFrame(df.groupby(['batsman'])['isSix'].sum()).reset_index().rename(columns ={'isSix':'sixes'})
  df=pd.merge(innings,runs,on ='batsman').merge(balls,on='batsman').merge(dismissals,on='batsman').merge(dots,on='batsman').merge(fours,on='batsman').merge(sixes,on='batsman')

  #StrikeRate
  df['SR'] = df.apply(lambda x: 100*(x['runs']/x['ballsFaced']), axis = 1)
  #runs per innings
  df['RPI'] = df.apply(lambda x: x['runs']/x['innings'], axis = 1)

  #balls per dismissals
  df['BPD'] = df.apply(lambda x: balls_per_dismissal(x['ballsFaced'], x['dismissals']), axis = 1)

  #balls per boundary
  df['BPB'] = df.apply(lambda x: balls_per_boundary(x['ballsFaced'], (x['fours'] + x['sixes'])), axis = 1)

  return df

In [ ]:
df1.head(2)

,matchId,inning,over_ball,over,ball,batting_team,bowling_team,batsman,non_striker,bowler,batsman_runs,extras,isWide,isNoBall,Byes,LegByes,Penalty,dismissal_kind,player_dismissed,Batting_Phase
0,1082591,1,0.1,0,1,Sunrisers Hyderabad,Royal Challengers Bangalore,DA Warner,S Dhawan,TS Mills,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Powerplay
1,1082591,1,0.2,0,2,Sunrisers Hyderabad,Royal Challengers Bangalore,DA Warner,S Dhawan,TS Mills,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Powerplay


In [ ]:
df2.head(2)

,outcome,event,date,gender,reserve_umpire,team1,toss_winner,umpire2,match_referee,season,team2,winner_runs,balls_per_over,player_of_match,date1,neutralvenue,venue,method,toss_decision,city,tv_umpire,date2,winner,umpire1,match_number,eliminator,winner_wickets,matchId
0,NaN,Indian Premier League,2017-04-05,male,N Pandit,Sunrisers Hyderabad,Royal Challengers Bangalore,NJ Llong,J Srinath,2017,Royal Challengers Bangalore,35.0,6,Yuvraj Singh,NaN,NaN,"Rajiv Gandhi International Stadium, Uppal",NaN,field,Hyderabad,A Deshmukh,NaN,Sunrisers Hyderabad,AY Dandekar,1.0,NaN,NaN,1082591
1,NaN,Indian Premier League,2017-04-06,male,Navdeep Singh,Rising Pune Supergiant,Rising Pune Supergiant,S Ravi,M Nayyar,2017,Mumbai Indians,NaN,6,SPD Smith,NaN,NaN,Maharashtra Cricket Association Stadium,NaN,field,Pune,VK Sharma,NaN,Rising Pune Supergiant,A Nand Kishore,2.0,NaN,7.0,1082592


In [ ]:
comb=pd.merge(df1,df2,on='matchId')

In [ ]:
comb.head(2)

,matchId,inning,over_ball,over,ball,batting_team,bowling_team,batsman,non_striker,bowler,batsman_runs,extras,isWide,isNoBall,Byes,LegByes,Penalty,dismissal_kind,player_dismissed,Batting_Phase,outcome,event,date,gender,reserve_umpire,team1,toss_winner,umpire2,match_referee,season,team2,winner_runs,balls_per_over,player_of_match,date1,neutralvenue,venue,method,toss_decision,city,tv_umpire,date2,winner,umpire1,match_number,eliminator,winner_wickets
0,1082591,1,0.1,0,1,Sunrisers Hyderabad,Royal Challengers Bangalore,DA Warner,S Dhawan,TS Mills,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Powerplay,NaN,Indian Premier League,2017-04-05,male,N Pandit,Sunrisers Hyderabad,Royal Challengers Bangalore,NJ Llong,J Srinath,2017,Royal Challengers Bangalore,35.0,6,Yuvraj Singh,NaN,NaN,"Rajiv Gandhi International Stadium, Uppal",NaN,field,Hyderabad,A Deshmukh,NaN,Sunrisers Hyderabad,AY Dandekar,1.0,NaN,NaN
1,1082591,1,0.2,0,2,Sunrisers Hyderabad,Royal Challengers Bangalore,DA Warner,S Dhawan,TS Mills,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Powerplay,NaN,Indian Premier League,2017-04-05,male,N Pandit,Sunrisers Hyderabad,Royal Challengers Bangalore,NJ Llong,J Srinath,2017,Royal Challengers Bangalore,35.0,6,Yuvraj Singh,NaN,NaN,"Rajiv Gandhi International Stadium, Uppal",NaN,field,Hyderabad,A Deshmukh,NaN,Sunrisers Hyderabad,AY Dandekar,1.0,NaN,NaN


In [ ]:
def ByCustom(df,current_venue,current_phase,current_opposition):
  df = df[df['venue'] == current_venue]
  df = df[df['Batting_Phase'] == current_phase]
  df = df[df.bowling_team == current_opposition]

  df.reset_index(inplace = True,drop = True)

  df['isDot'] = df['batsman_runs'].apply(lambda x : 1 if x == 0 else 0)
  df['isOne'] = df['batsman_runs'].apply(lambda x : 1 if x == 1 else 0)
  df['isTwo'] = df['batsman_runs'].apply(lambda x : 1 if x == 2 else 0)
  df['isThree'] = df['batsman_runs'].apply(lambda x : 1 if x == 3 else 0)
  df['isFour'] = df['batsman_runs'].apply(lambda x :1 if x == 4 else 0)
  df['isSix'] = df['batsman_runs'].apply(lambda x : 1 if x == 6 else 0)

  runs = pd.DataFrame(df.groupby(df['batsman'])['batsman_runs'].sum()).reset_index().rename(columns ={'batsman_runs':'runs'})
  balls = pd.DataFrame(df.groupby(['batsman'])['batsman_runs'].count()).reset_index().rename(columns = {'batsman_runs':'ballsFaced'})
  dismissals = pd.DataFrame(df.groupby(['batsman'])['player_dismissed'].count()).reset_index().rename(columns ={'player_dismissed':'dismissals'})
  innings = pd.DataFrame(df.groupby(['batsman'])['matchId'].apply(lambda x : len(list(np.unique(x))))).reset_index().rename(columns = {'matchId':'innings'})
  dots = pd.DataFrame(df.groupby(['batsman'])['isDot'].sum()).reset_index().rename(columns ={'isDot':'dots'})
  fours = pd.DataFrame(df.groupby(['batsman'])['isFour'].sum()).reset_index().rename(columns ={'isFour':'fours'})
  sixes = pd.DataFrame(df.groupby(['batsman'])['isSix'].sum()).reset_index().rename(columns ={'isSix':'sixes'})
  df=pd.merge(innings,runs,on ='batsman').merge(balls,on='batsman').merge(dismissals,on='batsman').merge(dots,on='batsman').merge(fours,on='batsman').merge(sixes,on='batsman')

  #StrikeRate
  df['SR'] = df.apply(lambda x: 100*(x['runs']/x['ballsFaced']), axis = 1)
  #runs per innings
  df['RPI'] = df.apply(lambda x: x['runs']/x['innings'], axis = 1)

  #balls per dismissals
  df['BPD'] = df.apply(lambda x: balls_per_dismissal(x['ballsFaced'], x['dismissals']), axis = 1)

  #balls per boundary
  df['BPB'] = df.apply(lambda x: balls_per_boundary(x['ballsFaced'], (x['fours'] + x['sixes'])), axis = 1)

  return df

In [ ]:
req_df = ByCustom(comb,'MA Chidambaram Stadium, Chepauk','Middle','Chennai Super Kings')


In [ ]:
req_df['dot_percentage'] = req_df.apply(lambda x : x['dots']/x['ballsFaced'],axis=1)


In [ ]:
req_df.head(2)

,batsman,innings,runs,ballsFaced,dismissals,dots,fours,sixes,SR,RPI,BPD,BPB,dot_percentage
0,A Ashish Reddy,1,1,2,0,1,0,0,50.000000,1.0,2.0,2.0,0.500000
1,A Mishra,1,6,11,1,6,0,0,54.545455,6.0,11.0,11.0,0.545455


In [ ]:
wt_sr,wt_rpi,wt_bpd,wt_dot_percentage = 0.13,0.27,0.16,0.45


In [ ]:
req_df = req_df[(req_df['ballsFaced']>=25) & (req_df['innings']>=2)]


In [ ]:
req_df=req_df.reset_index(drop=True)

### Calculation - Normalizing all values to same dimensions

In [ ]:
#step1: square of all values
req_df['calc_SR'] = req_df['SR'].apply(lambda x: x*x)
req_df['calc_RPI'] = req_df['RPI'].apply(lambda x: x*x)
req_df['calc_BPD'] = req_df['BPD'].apply(lambda x: x*x)
req_df['calc_dot_percentage'] = req_df['dot_percentage'].apply(lambda x: x*x)

sq_sr, sq_rpi, sq_bpd, sq_dot_percentage = np.sqrt(req_df[['calc_SR','calc_RPI', 'calc_BPD', 'calc_dot_percentage']].sum(axis = 0))

req_df['calc_SR'] = req_df['calc_SR'].apply(lambda x: x/sq_sr)
req_df['calc_RPI'] = req_df['calc_RPI'].apply(lambda x: x/sq_rpi)
req_df['calc_BPD'] = req_df['calc_BPD'].apply(lambda x: x/sq_bpd)
req_df['calc_dot_percentage'] = req_df['calc_dot_percentage'].apply(lambda x: x/sq_dot_percentage)

req_df['calc_SR'] = req_df['calc_SR'].apply(lambda x: x*wt_sr)
req_df['calc_RPI'] = req_df['calc_RPI'].apply(lambda x: x*wt_rpi)
req_df['calc_BPD'] = req_df['calc_BPD'].apply(lambda x: x*wt_bpd)
req_df['calc_dot_percentage'] = req_df['calc_dot_percentage'].apply(lambda x: x*wt_dot_percentage)

best_sr, worst_sr = max(req_df['calc_SR']), min(req_df['calc_SR'])
best_rpi, worst_rpi = max(req_df['calc_RPI']), min(req_df['calc_RPI'])
best_bpd, worst_bpd = max(req_df['calc_BPD']), min(req_df['calc_BPD'])
best_dot_percentage, worst_dot_percentage = min(req_df['calc_dot_percentage']), max(req_df['calc_dot_percentage'])


###  Calculation - Comparision against the best & worst values


In [ ]:
req_df['dev_best_SR'] = req_df['calc_SR'].apply(lambda x: (x-best_sr)*(x-best_sr))
req_df['dev_best_RPI'] = req_df['calc_RPI'].apply(lambda x: (x-best_rpi)*(x-best_rpi))
req_df['dev_best_BPD'] = req_df['calc_BPD'].apply(lambda x: (x-best_bpd)*(x-best_bpd))
req_df['dev_best_dot_percentage'] = req_df['calc_dot_percentage'].apply(lambda x: (x-best_dot_percentage)*(x-best_dot_percentage))

req_df['dev_best_sqrt'] = req_df.apply(lambda x: x['dev_best_SR'] + x['dev_best_RPI'] + x['dev_best_BPD'] + x['dev_best_dot_percentage'], axis = 1)

req_df['dev_worst_SR'] = req_df['calc_SR'].apply(lambda x: (x-worst_sr)*(x-worst_sr))
req_df['dev_worst_RPI'] = req_df['calc_RPI'].apply(lambda x: (x-worst_rpi)*(x-worst_rpi))
req_df['dev_worst_BPD'] = req_df['calc_BPD'].apply(lambda x: (x-worst_bpd)*(x-worst_bpd))
req_df['dev_worst_dot_percentage'] = req_df['calc_dot_percentage'].apply(lambda x: (x-worst_dot_percentage)*(x-worst_dot_percentage))

req_df['dev_worst_sqrt'] = req_df.apply(lambda x: x['dev_worst_SR'] + x['dev_worst_RPI'] + x['dev_worst_BPD'] + x['dev_worst_dot_percentage'], axis = 1)


### Final Step - Overall score for each player ( values from 0 - 1)


In [ ]:
req_df['score'] = req_df.apply(lambda x: x['dev_worst_sqrt']/(x['dev_worst_sqrt'] + x['dev_best_sqrt']), axis = 1)

In [ ]:
req_df[['batsman', 'score']].head()


,batsman,score
0,AB de Villiers,0.814600
1,AD Mathews,0.248822
2,AL Menaria,0.010808
3,AT Rayudu,0.000873
4,CL White,0.089106


### End Result: Top players for our required role


In [ ]:
req_df[[ 'batsman','innings', 'runs', 'ballsFaced', 'dismissals', 'dot_percentage', 'score']].sort_values(['score'], ascending = False).reset_index(drop = True).head(25)


,batsman,innings,runs,ballsFaced,dismissals,dot_percentage,score
0,SE Marsh,3,93,50,1,0.240000,0.902625
1,AB de Villiers,3,99,67,0,0.253731,0.814600
2,NV Ojha,3,67,41,2,0.292683,0.514822
3,SR Watson,4,105,66,3,0.287879,0.507664
4,G Gambhir,3,80,72,1,0.319444,0.506511
5,MS Bisla,3,82,57,2,0.280702,0.388337
6,AD Mathews,2,52,43,0,0.302326,0.248822
7,Y Venugopal Rao,3,65,49,2,0.326531,0.218426
8,V Kohli,7,172,137,5,0.299270,0.193650
9,DA Miller,3,47,36,2,0.277778,0.162812
